In [137]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme
import re

In [138]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [160]:
data_exp = pandas.read_csv("./datasets/PND32_abundances.txt",sep='\t',index_col=0)
data_pred = pandas.read_csv("./datasets/2.3.1.high_gamma_tail_scores.csv",index_col=0)[["low_iron"]]

### Parse data

In [161]:
def parse(x):
    return float(re.findall("\d\.\d{2,3}",x)[0])
data_exp = data_exp.applymap(lambda x:parse(x))

In [162]:
high = (data_exp["ID"] > data_exp["Control"]) & (data_exp["pvalue"] < 0.05)
low = (data_exp["ID"] < data_exp["Control"]) & (data_exp["pvalue"] < 0.05)

### Make comparable

In [163]:
dct = {}
for i,row in data_exp.iterrows():
    if high[i]:
        dct[i] = 1
    elif low[i]:
        dct[i] = -1
    else:
        dct[i] = 0
experimental = pandas.DataFrame.from_dict({"experimental":dct})
# experimental = experimental[experimental["experimental"] != 0]

In [164]:
dct = {}
for i,row in data_pred.iterrows():
    if row["low_iron"] > 0 :
        dct[i] = 1
    elif row["low_iron"] < 0 :
        dct[i] = -1
    else:
        dct[i] = 0
predicted = pandas.DataFrame.from_dict({"predicted":dct})

In [165]:
df = pandas.concat([predicted,experimental,data_exp,data_pred],axis=1).dropna()

In [166]:
tp = df[(df["predicted"] == 1.0) & (df["predicted"] == df["experimental"])].shape[0]
tn = df[(df["predicted"] == -1.0) & (df["predicted"] == df["experimental"])].shape[0]
fp = df[(df["predicted"] == 1.0) & (df["predicted"] != df["experimental"])].shape[0]
fn = df[(df["predicted"] == -1.0) & (df["predicted"] != df["experimental"])].shape[0]
tp,tn,fp,fn

(2, 4, 6, 16)